<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modelling" data-toc-modified-id="Modelling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modelling</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Utilities" data-toc-modified-id="Utilities-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Utilities</a></span></li></ul></li><li><span><a href="#Optimizing-top-10-Category" data-toc-modified-id="Optimizing-top-10-Category-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Optimizing top 10 Category</a></span><ul class="toc-item"><li><span><a href="#Training-Model" data-toc-modified-id="Training-Model-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Training Model</a></span></li><li><span><a href="#Price-Optimization-Demonstration" data-toc-modified-id="Price-Optimization-Demonstration-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Price Optimization Demonstration</a></span></li></ul></li></ul></li></ul></div>

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm
import joblib

import datetime
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_row', 500)

In [2]:
data = pd.read_csv('../data/prepared/interpolated_demand_history_by_category.csv', index_col=0)
data['order_purchase_timestamp'] = pd.to_datetime(data['order_purchase_timestamp'])
data

,product_category_name,order_purchase_timestamp,sales,price
0,furniture_decor,2016-09-04,2,111285.5364
1,telephony,2016-09-05,1,181684.4400
2,furniture_decor,2016-09-05,1,122110.2648
3,furniture_decor,2016-09-06,1,122110.2648
4,furniture_decor,2016-09-07,1,122110.2648
...,...,...,...,...
37802,kitchen_dining_laundry_garden_furniture,2018-08-30,1,442760.4000
37803,kitchen_dining_laundry_garden_furniture,2018-08-31,1,442760.4000
37804,kitchen_dining_laundry_garden_furniture,2018-09-01,1,442760.4000
37805,kitchen_dining_laundry_garden_furniture,2018-09-02,1,442760.4000


In [3]:
selected_categories = ['bed_bath_table', 'health_beauty', 'sports_leisure', 'furniture_decor','computers_accessories', 'housewares', 'watches_gifts', 'telephony','garden_tools', 'auto']
data = data[data.product_category_name.apply(lambda x: x in selected_categories)]

In [4]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[['sales', 'price']])

encoder = LabelEncoder()
encoded_category = encoder.fit_transform(data[['product_category_name']])

/home/manfred/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
joblib.dump(scaler, "../model/demand_forecasting/scaler.joblib")
joblib.dump(encoder, "../model/demand_forecasting/encoder.joblib")

['../model/demand_forecasting/encoder.joblib']

In [5]:
data['scaled_sales'] = scaled_data[:,0]
data['scaled_price'] = scaled_data[:,1]
data['encoded_category'] = encoded_category
data

/tmp/ipykernel_1863/4205915431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['scaled_sales'] = scaled_data[:,0]
/tmp/ipykernel_1863/4205915431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['scaled_price'] = scaled_data[:,1]
/tmp/ipykernel_1863/4205915431.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,product_category_name,order_purchase_timestamp,sales,price,scaled_sales,scaled_price,encoded_category
0,furniture_decor,2016-09-04,2,111285.5364,-1.233255,-0.775227,3
1,telephony,2016-09-05,1,181684.4400,-1.240183,-0.598523,8
2,furniture_decor,2016-09-05,1,122110.2648,-1.240183,-0.748057,3
3,furniture_decor,2016-09-06,1,122110.2648,-1.240183,-0.748057,3
4,furniture_decor,2016-09-07,1,122110.2648,-1.240183,-0.748057,3
...,...,...,...,...,...,...,...
37791,computers_accessories,2018-08-28,4,281595.6144,-1.219400,-0.347743,2
37795,watches_gifts,2018-08-29,2,410393.0880,-1.233255,-0.024456,9
37796,computers_accessories,2018-08-29,1,259518.6648,-1.240183,-0.403157,2
37798,health_beauty,2018-08-29,3,206061.7080,-1.226328,-0.537336,5


# Modelling

### Utilities

In [18]:
# !pip3 install wandb

import wandb
from wandb.keras import WandbCallback
import tensorflow as tf

In [19]:
def show_sample(ds, n_sample=3):
    for x in ds.take(n_sample):
        print(x)

In [20]:
def show_history(history):
    fig = plt.figure(figsize=(14, 8))
    plt.plot(pd.DataFrame(history.history)[['loss', 'val_loss']])

## Optimizing top 10 Category

In [21]:
wandb.init(project="cateogory-discount-optimization", entity="anakbangkit", reinit=True)
config = wandb.config

wandb: Currently logged in as: manfredmichael (anakbangkit). Use `wandb login --relogin` to force relogin
2022-06-15 06:53:26.833581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 06:53:26.833628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Training Model

In [22]:
config.test_size = 0.2
config.valid_size = 0.2
config.window_size = 30
config.batch_size = 8
config.buffer_size = 100_000
config.stateful = False

In [23]:
data = data.sort_values('order_purchase_timestamp')
data

,product_category_name,order_purchase_timestamp,sales,price,scaled_sales,scaled_price,encoded_category
0,furniture_decor,2016-09-04,2,111285.5364,-1.233255,-0.775227,3
1,telephony,2016-09-05,1,181684.4400,-1.240183,-0.598523,8
2,furniture_decor,2016-09-05,1,122110.2648,-1.240183,-0.748057,3
3,furniture_decor,2016-09-06,1,122110.2648,-1.240183,-0.748057,3
4,furniture_decor,2016-09-07,1,122110.2648,-1.240183,-0.748057,3
...,...,...,...,...,...,...,...
37784,watches_gifts,2018-08-28,2,241228.0800,-1.233255,-0.449067,9
37796,computers_accessories,2018-08-29,1,259518.6648,-1.240183,-0.403157,2
37798,health_beauty,2018-08-29,3,206061.7080,-1.226328,-0.537336,5
37795,watches_gifts,2018-08-29,2,410393.0880,-1.233255,-0.024456,9


In [24]:
selected_categories = ['bed_bath_table', 'health_beauty', 'sports_leisure', 'furniture_decor','computers_accessories', 'housewares', 'watches_gifts', 'telephony','garden_tools', 'auto']
data = data[data.product_category_name.apply(lambda x: x in selected_categories)]

In [25]:
data

,product_category_name,order_purchase_timestamp,sales,price,scaled_sales,scaled_price,encoded_category
0,furniture_decor,2016-09-04,2,111285.5364,-1.233255,-0.775227,3
1,telephony,2016-09-05,1,181684.4400,-1.240183,-0.598523,8
2,furniture_decor,2016-09-05,1,122110.2648,-1.240183,-0.748057,3
3,furniture_decor,2016-09-06,1,122110.2648,-1.240183,-0.748057,3
4,furniture_decor,2016-09-07,1,122110.2648,-1.240183,-0.748057,3
...,...,...,...,...,...,...,...
37784,watches_gifts,2018-08-28,2,241228.0800,-1.233255,-0.449067,9
37796,computers_accessories,2018-08-29,1,259518.6648,-1.240183,-0.403157,2
37798,health_beauty,2018-08-29,3,206061.7080,-1.226328,-0.537336,5
37795,watches_gifts,2018-08-29,2,410393.0880,-1.233255,-0.024456,9


In [46]:
def train_val_split(data):
    train_split_time = int((1 - (config.valid_size + config.test_size)) * len(data))
    valid_split_time = int((1 - config.test_size) * len(data))
    
    train = data.iloc[:train_split_time]
    valid = data.iloc[train_split_time:valid_split_time]
    test = data.iloc[valid_split_time:]

    return train, valid, test



In [47]:
class DataPreprocessing:
    def __init__(self, window_size, batch_size, buffer_size, stateful):
        self.scaler = StandardScaler()
        self.window_size = window_size
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        self.stateful = stateful
#         scaled_data = scaler.fit_transform(data[['sales', 'price']])

    def fit(self, data):
        self.scaler.fit(data[['sales', 'price']])
        
    def rescale(self, data):
        data = data.copy()
        scaled_data = self.scaler.transform(data[['sales', 'price']])
        data['scaled_sales'] = scaled_data[:,0]
        data['scaled_price'] = scaled_data[:,1]
        return data
    
    def train_val_split(self, data):
        train_split_time = int((1 - (config.valid_size + config.test_size)) * len(data))
        valid_split_time = int((1 - config.test_size) * len(data))

        train = data.iloc[:train_split_time]
        valid = data.iloc[train_split_time:valid_split_time]
        test = data.iloc[valid_split_time:]

        return train, valid, test
    
    def create_windowed_dataset(self, data, with_label=True, rescale=True):
        if rescale:
            data = self.rescale(data)
        
        data = data[['scaled_sales', 'scaled_price', 'encoded_category', 'sales']].values
        ds = tf.data.Dataset.from_tensor_slices(data)
        ds = ds.window(self.window_size + 1, shift=1, drop_remainder=True)
        ds = ds.flat_map(lambda w: w.batch(self.window_size + 1))

        if not self.stateful:
            ds = ds.shuffle(self.buffer_size)

        if with_label:
            ds = ds.map(lambda w: ((w[:-1, :2], w[-1:, 1], w[-1, 2]), w[-1:, 3]))
        else:
            ds = ds.map(lambda w: (w[:-1, :2], w[-1:, 1], w[-1, 2]))

        ds = ds.batch(self.batch_size, drop_remainder=self.stateful)
        if not with_label:
            ds = ds.prefetch(1)

        return ds
    
    def preprocess(self, data):
        train_datasets_list = []
        valid_datasets_list = []
        test_datasets_list = []
        for category, group in tqdm(data.groupby('product_category_name')):
            train, valid, test = self.train_val_split(group)
            train_datasets_list.append(self.create_windowed_dataset(train))
            valid_datasets_list.append(self.create_windowed_dataset(valid))
            test_datasets_list.append(self.create_windowed_dataset(test))
            
        train_datasets = train_datasets_list[0]
        valid_datasets = valid_datasets_list[0]
        test_datasets = test_datasets_list[0]
        for train, valid, test in zip(train_datasets_list[1:], valid_datasets_list[1:], test_datasets_list[1:]):
            train_datasets = train_datasets.concatenate(train)
            valid_datasets = valid_datasets.concatenate(valid)
            test_datasets = test_datasets.concatenate(test)
            
        return train_datasets, valid_datasets, test_datasets

In [28]:
# joblib.dump(DataPreprocessing, "../model/demand_forecasting/DataPreprocessing.joblib")

In [29]:
# DataPreprocessing = joblib.load("../model/demand_forecasting/DataPreprocessing.joblib")

In [30]:
data_pipeline = DataPreprocessing(
    window_size = config.window_size,
    batch_size = config.batch_size,
    buffer_size = config.buffer_size,
    stateful = config.stateful)

In [31]:
data_pipeline.fit(data)

In [49]:
train, valid, test = train_val_split(data)

In [50]:
train_set, valid_set, test_set = data_pipeline.preprocess(data)

100%|███████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.94it/s]


In [34]:
show_sample(train, n_sample=2)

((<tf.Tensor: shape=(8, 30, 2), dtype=float64, numpy=
array([[[-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+00, -8.25046168e-01],
        [-1.23325515e+

In [ ]:
config.rnn_unit = 64
config.rnn_layer = 3
config.fc1_unit = 128
config.fc2_unit = 64
config.dropout_rate = 0.2
config.embedding_dim = 16

config.batch_normalization = False
config.use_bias = True

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Input, GRU, LSTM, Concatenate, BatchNormalization, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def build_model(rnn_unit, fc1_unit, fc2_unit, embedding_dim, dropout_rate=0, rnn_layer=0, stateful=False, batch_normalization=False):
    if stateful:
        history_input = Input(batch_shape=(config.batch_size, config.window_size, 2), name='history_input')
    else:
        history_input = Input(shape=(config.window_size, 2))
    price_input = Input(shape=(1), name='price_input')
    category_input = Input(shape=(1), name='category_input')
    category = Embedding(10, embedding_dim, name='category_embedding')(category_input)
    category = Flatten()(category)
    
#     if config.batch_normalization:
#         history_input = BatchNormalization()(history_input)
#         price_input = BatchNormalization()(price_input)
        
    rnn = history_input
    for i in range(rnn_layer-1):
        rnn = LSTM(rnn_unit, use_bias=config.use_bias, return_sequences=True, dropout=dropout_rate, stateful=stateful)(rnn)
    if rnn_layer > 0:
        rnn = LSTM(rnn_unit, use_bias=config.use_bias, dropout=dropout_rate, stateful=stateful)(rnn)
    else:
        rnn = Flatten()(rnn)
        
    
    concat_ = Concatenate()([rnn, price_input, category])
    if dropout_rate > 0:
        concat_ = Dropout(dropout_rate)(concat_)
    fc = Dense(fc1_unit, activation='relu', use_bias=config.use_bias)(concat_)
    
    if dropout_rate > 0:
        fc = Dropout(dropout_rate)(fc)
    fc = Dense(fc2_unit, activation='relu', use_bias=config.use_bias)(fc)
    
    if dropout_rate > 0:
        fc = Dropout(dropout_rate)(fc)
    output = Dense(1, use_bias=config.use_bias, name='sales_output')(fc)
    
    model = Model([history_input, price_input, category_input], output)
    return model

In [25]:
model = build_model(
    rnn_unit=config.rnn_unit,
    rnn_layer=config.rnn_layer,
    fc1_unit=config.fc1_unit,
    fc2_unit=config.fc2_unit,
    stateful=config.stateful,
    batch_normalization=config.batch_normalization,
    dropout_rate=config.dropout_rate,
    embedding_dim=config.embedding_dim,
)

In [26]:
model.compile(
    optimizer=Adam(),
    loss='mse',
)

In [27]:
earlystopping_cb = EarlyStopping(
    monitor="loss",
    patience=100,
    restore_best_weights=True,
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='../model/demand_forecasting',
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
)

In [ ]:
history = model.fit(
    train,
    validation_data=valid,
    epochs=1000,
    callbacks=[WandbCallback(), earlystopping_cb, checkpoint_cb],
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/1000
    491/Unknown - 39s 50ms/step - loss: 13567.2617

2022-06-14 19:32:34.360684: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../model/demand_forecasting/assets


INFO:tensorflow:Assets written to: ../model/demand_forecasting/assets


...

INFO:tensorflow:Assets written to: ../model/demand_forecasting/assets


INFO:tensorflow:Assets written to: ../model/demand_forecasting/assets


491/491 [==============================] - 83s 169ms/step - loss: 210.6549 - val_loss: 138.5595 - _timestamp: 1655224630.0000 - _runtime: 14761.0000
Epoch 608/1000
491/491 [==============================] - 23s 36ms/step - loss: 210.8791 - val_loss: 764.0000 - _timestamp: 1655224723.0000 - _runtime: 14854.0000
Epoch 609/1000
491/491 [==============================] - 18s 36ms/step - loss: 222.2538 - val_loss: 559.3895 - _timestamp: 1655224741.0000 - _runtime: 14872.0000
Epoch 610/1000
491/491 [==============================] - 17s 36ms/step - loss: 223.8761 - val_loss: 932.2905 - _timestamp: 1655224759.0000 - _runtime: 14890.0000
Epoch 611/1000
491/491 [==============================] - 18s 36ms/step - loss: 219.2344 - val_loss: 1158.6017 - _timestamp: 1655224776.0000 - _runtime: 14907.0000
Epoch 612/1000
491/491 [==============================] - 18s 36ms/step - loss: 219.4386 - val_loss: 1600.5128 - _timestamp: 1655224794.0000 - _runtime: 14925.0000
Epoch 613/1000
491/491 [==========

491/491 [==============================] - 17s 36ms/step - loss: 221.5855 - val_loss: 533.3320 - _timestamp: 1655226479.0000 - _runtime: 16610.0000
Epoch 708/1000
491/491 [==============================] - 18s 36ms/step - loss: 213.1219 - val_loss: 849.6878 - _timestamp: 1655226496.0000 - _runtime: 16627.0000
Epoch 709/1000
491/491 [==============================] - 18s 36ms/step - loss: 206.0623 - val_loss: 747.7977 - _timestamp: 1655226514.0000 - _runtime: 16645.0000
Epoch 710/1000
491/491 [==============================] - 18s 36ms/step - loss: 209.8226 - val_loss: 1468.6024 - _timestamp: 1655226532.0000 - _runtime: 16663.0000
Epoch 711/1000
491/491 [==============================] - 17s 36ms/step - loss: 196.3568 - val_loss: 1364.8766 - _timestamp: 1655226550.0000 - _runtime: 16681.0000
Epoch 712/1000
491/491 [==============================] - 17s 36ms/step - loss: 210.4371 - val_loss: 450.8384 - _timestamp: 1655226567.0000 - _runtime: 16698.0000
Epoch 713/1000
491/491 [===========

491/491 [==============================] - 18s 36ms/step - loss: 191.8564 - val_loss: 1806.9581 - _timestamp: 1655228255.0000 - _runtime: 18386.0000
Epoch 808/1000
491/491 [==============================] - 18s 36ms/step - loss: 198.5390 - val_loss: 523.0257 - _timestamp: 1655228273.0000 - _runtime: 18404.0000
Epoch 809/1000
491/491 [==============================] - 18s 36ms/step - loss: 203.8027 - val_loss: 811.7017 - _timestamp: 1655228291.0000 - _runtime: 18422.0000
Epoch 810/1000
170/491 [=========>....................] - ETA: 10s - loss: 276.5261

In [ ]:
show_history(history)

In [ ]:
print(test_bruh)

### Price Optimization Demonstration

In [35]:
def predict_next_week(model, data):
    pred = model.predict(data)[0][0]
    
    return pred

def forecast(model, data, prices, window_size=config.window_size, batch_size=config.batch_size, buffer_size=config.buffer_size):
    data = data[['scaled_sales', 'scaled_price', 'sales']].copy()
    predictions = []
    for price in prices:
        data = data.append({'scaled_sales':0, 'scaled_price': scaler.transform([[0, price]])[0, 1]}, ignore_index = True)
        current_week = create_windowed_dataset(data.iloc[-(config.batch_size + config.window_size):], with_label=True)
        next_week_sales = predict_next_week(model, current_week)
        predictions.append(next_week_sales)
        data['sales'].iloc[-1] = next_week_sales
        
    return predictions


In [36]:
model = tf.keras.models.load_model("../model/demand_forecasting")

In [ ]:
# class ModelPipeline:
#     def __init__(self, model, data_pipeline):
#         self.model = model
#         self.data_pipeline = data_pipeline
        
#     def predict_next_week(self, model, data):
#         return self.model.predict(data)[0][0]

#     def forecast(self, data, prices):
#         data = self.data_pipeline.rescale(data)
#         data = data[['scaled_sales', 'scaled_price', 'sales']].copy()
#         predictions = []
#         for price in prices:
#             data = data.append({'scaled_sales':0, 'scaled_price': self.data_pipeline.scaler.transform([[0, price]])[0, 1]}, ignore_index = True)
#             current_week = self.data_pipeline.create_windowed_dataset(data.iloc[-(self.data_pipeline.batch_size + self.data_pipeline.window_size):], with_label=True, rescale=False)
#             next_week_sales = self.predict_next_week(self.model, current_week)
#             predictions.append(next_week_sales)
#             data['sales'].iloc[-1] = next_week_sales

#         return predictions

In [ ]:
pipeline = ModelPipeline(model, data_pipeline)

In [ ]:
joblib.dump(pipeline, "../model/demand_forecasting/pipeline.joblib")

In [ ]:
predicted_sales = pipeline.forecast(valid, [312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 
                                         ])
plt.plot(predicted_sales)
print(np.mean(predicted_sales))

In [ ]:
# No discount at all
# predicted_sales = forecast(model, valid, [312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
#                                           312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
#                                           312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355,
#                                           312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 
#                                          ])
plt.plot(predicted_sales)
print(np.mean(predicted_sales))

In [51]:
# No discount at all
predicted_sales = forecast(model, valid, [312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 300000.685355, 300000.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 300000.685355, 300000.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 300000.685355, 300000.685355,
                                          312089.685355, 312089.685355, 312089.685355, 312089.685355, 312089.685355, 300000.685355, 300000.685355,
                                         ])
plt.plot(predicted_sales)
print(np.mean(predicted_sales))

NameError: name 'create_windowed_dataset' is not defined

In [ ]:
# Discount on the first 2 weeks
predicted_sales = forecast(model, valid, [0, 0, 0, 0, 0, 0, 0,
                                          0, 0, 0, 0, 0, 0, 0,
                                          0, 0, 0, 0, 0, 0, 0,
                                          0, 0, 0, 0, 0, 0, 0,
                                         ])
plt.plot(predicted_sales)
print(np.mean(predicted_sales))

In [ ]:
valid